In [ ]:
vectorstore = None

def ask_question(question: str):
    global vectorstore

    if vectorstore is None:
        return " Upload a document first.", []

    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    docs = retriever.invoke(question)

    if not docs:
        return "I couldn't find relevant information in the document.", []

    context = "\n\n".join([d.page_content for d in docs])
    citations = [f"Page {d.metadata.get('page', '?')}" for d in docs]

    history = get_history()
    history_text = "\n".join([f"Q: {h['q']} A: {h['a']}" for h in history])

    messages = [
        {
            "role": "system",
            "content": (
                "You are a legal assistant. Use the context provided to answer questions thoroughly. "
                "Be concise but complete. If information is not in the context, say so explicitly.\n\n"
                f"Conversation History:\n{history_text}\n\n"
                f"Context:\n{context}"
            )
        },
        {"role": "user", "content": question}
    ]

    answer = ask_llm(messages)
    add_to_history(question, answer)

    return answer, citations

In [ ]:
if __name__ == "__main__":
    import threading

    def run_api():
        uvicorn.run(app, host="0.0.0.0", port=8000)

    threading.Thread(target=run_api).start()
    launch_ui()